<a href="https://colab.research.google.com/github/rsaran-BioAI/AGILE/blob/main/ConnectionAware_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connecting the drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#%%bash
#MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
#MINICONDA_PREFIX=/usr/local
#wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
#chmod +x $MINICONDA_INSTALLER_SCRIPT
#./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Process is interrupted.


In [2]:
!pip install rdkit

In [3]:
!pip install torch_geometric

In [4]:
!pip install torch

In [5]:
!pip install guacamol

In [6]:
!pip install tensorboardX

In [7]:
!pip install networkx

## Merging Operation Learning

In [8]:
%cd /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/src/

/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/src


In [9]:
import multiprocessing as mp
import os
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from multiprocessing import Process, Queue
from typing import Dict, List, Tuple

import networkx as nx
from rdkit import Chem

In [10]:
import arguments
import model.mydataclass
from arguments import parse_arguments
from model.mydataclass import Paths

In [11]:
@dataclass
class MolGraph:
    idx: int
    mol_graph: Chem.rdchem.Mol
    merging_graph: nx.Graph

    def __init__(self, smiles: str, idx: int=0) -> "MolGraph":
        self.idx = idx
        self.mol_graph = Chem.MolFromSmiles(smiles)
        self.merging_graph = nx.Graph(Chem.rdmolops.GetAdjacencyMatrix(self.mol_graph))
        for atom in self.mol_graph.GetAtoms():
            self.merging_graph.nodes[atom.GetIdx()]["atom_indices"] = set([atom.GetIdx()])

    def apply_merging_operation(self, motif: str, stats: Dict[str, int], indices: Dict[str, Dict[int, int]]) -> None:
        if self.merging_graph.number_of_nodes() == 1:
            return
        new_graph = self.merging_graph.copy()
        for (node1, node2) in self.merging_graph.edges:
            if not new_graph.has_edge(node1, node2):
                continue
            atom_indices = new_graph.nodes[node1]["atom_indices"].union(new_graph.nodes[node2]["atom_indices"])
            motif_smiles = fragment2smiles(self, atom_indices)
            if motif_smiles == motif:
                graph_before_merge = new_graph.copy()
                merge_nodes(new_graph, node1, node2)
                update_stats(self, graph_before_merge, new_graph, node1, node2, stats, indices, self.idx)
        self.merging_graph = new_graph
        indices[motif][self.idx] = 0

    def apply_merging_operation_producer(self, motif: str, q: Queue) -> None:
        if self.merging_graph.number_of_nodes() == 1:
            return
        new_graph = self.merging_graph.copy()
        for (node1, node2) in self.merging_graph.edges:
            if not new_graph.has_edge(node1, node2):
                continue
            atom_indices = new_graph.nodes[node1]["atom_indices"].union(new_graph.nodes[node2]["atom_indices"])
            motif_smiles = fragment2smiles(self, atom_indices)
            if motif_smiles == motif:
                graph_before_merge = new_graph.copy()
                merge_nodes(new_graph, node1, node2)
                update_stats_producer(self, graph_before_merge, new_graph, node1, node2, q, self.idx)
        q.put((motif, self.idx, new_graph))

def load_batch_mols(batch: List[Tuple[int, str]]) -> List[MolGraph]:
    return [MolGraph(smi, idx) for (idx, smi) in batch]

def load_mols(train_path: str, num_workers: int) -> List[MolGraph]:
    print(f"[{datetime.now()}] Loading molecules...")
    smiles_list = [smi.strip("\n") for smi in open(train_path)]
    smiles_list = [(i, smi) for (i, smi) in enumerate(smiles_list)]

    batch_size = (len(smiles_list) - 1) // num_workers + 1
    batches = [smiles_list[i : i + batch_size] for i in range(0, len(smiles_list), batch_size)]
    mols: List[MolGraph]= []
    with mp.Pool(num_workers) as pool:
        for mols_batch in pool.imap(load_batch_mols, batches):
            mols.extend(mols_batch)

    print(f"[{datetime.now()}] Loading molecules finished. Total: {len(mols)} molecules.\n")
    return mols

def fragment2smiles(mol: MolGraph, indices: List[int]) -> str:
    smiles = Chem.MolFragmentToSmiles(mol.mol_graph, tuple(indices))
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles, sanitize=False))

def merge_nodes(graph: nx.Graph, node1: int, node2: int) -> None:
    neighbors = [n for n in graph.neighbors(node2)]
    atom_indices = graph.nodes[node1]["atom_indices"].union(graph.nodes[node2]["atom_indices"])
    for n in neighbors:
        if node1 != n and not graph.has_edge(node1, n):
            graph.add_edge(node1, n)
        graph.remove_edge(node2, n)
    graph.remove_node(node2)
    graph.nodes[node1]["atom_indices"] = atom_indices

def get_stats_producer(batch: List[MolGraph], q: Queue):
    for mol in batch:
        for (node1, node2) in mol.merging_graph.edges:
            atom_indices = mol.merging_graph.nodes[node1]["atom_indices"].union(mol.merging_graph.nodes[node2]["atom_indices"])
            motif_smiles = fragment2smiles(mol, atom_indices)
            q.put((mol.idx, motif_smiles))
    q.put(None)

def get_stats_consumer(stats: Dict[str, int], indices: Dict[str, Dict[int, int]], q: Queue, num_workers: int):
    num_tasks_done = 0
    while True:
        info = q.get()
        if info == None:
            num_tasks_done += 1
            if num_tasks_done == num_workers:
                break
        else:
            (idx, smi) = info
            stats[smi] += 1
            indices[smi][idx] += 1

def get_stats(mols: List[MolGraph], num_workers: int) -> Tuple[Dict[str, int], Dict[int, int]]:
    print(f"[{datetime.now()}] Begin getting statistics.")
    stats = defaultdict(int)
    indices = defaultdict(lambda: defaultdict(int))
    if num_workers == 1:
        for mol in mols:
            for (node1, node2) in mol.merging_graph.edges:
                atom_indices = mol.merging_graph.nodes[node1]["atom_indices"].union(mol.merging_graph.nodes[node2]["atom_indices"])
                motif_smiles = fragment2smiles(mol, atom_indices)
                stats[motif_smiles] += 1
                indices[motif_smiles][mol.idx] += 1
    else:
        batch_size = (len(mols) - 1) // num_workers + 1
        batches = [mols[i : i + batch_size] for i in range(0, len(mols), batch_size)]
        q = Queue()
        producers = [Process(target=get_stats_producer, args=(batches[i], q)) for i in range(num_workers)]
        [p.start() for p in producers]
        get_stats_consumer(stats, indices, q, num_workers)
        [p.join() for p in producers]
    return stats, indices

def update_stats(mol: MolGraph, graph: nx.Graph, new_graph: nx.Graph, node1: int, node2: int, stats: Dict[str, int], indices: Dict[str, Dict[int, int]], i: int):
    neighbors1 = [n for n in graph.neighbors(node1)]
    for n in neighbors1:
        if n != node2:
            atom_indices = graph.nodes[node1]["atom_indices"].union(graph.nodes[n]["atom_indices"])
            motif_smiles = fragment2smiles(mol, atom_indices)
            stats[motif_smiles] -= 1
            indices[motif_smiles][i] -= 1
    neighbors2 = [n for n in graph.neighbors(node2)]
    for n in neighbors2:
        if n != node1:
            atom_indices = graph.nodes[node2]["atom_indices"].union(graph.nodes[n]["atom_indices"])
            motif_smiles = fragment2smiles(mol, atom_indices)
            stats[motif_smiles] -= 1
            indices[motif_smiles][i] -= 1
    neighbors = [n for n in new_graph.neighbors(node1)]
    for n in neighbors:
        atom_indices = new_graph.nodes[node1]["atom_indices"].union(new_graph.nodes[n]["atom_indices"])
        motif_smiles = fragment2smiles(mol, atom_indices)
        stats[motif_smiles] += 1
        indices[motif_smiles][i] += 1

def update_stats_producer(mol: MolGraph, graph: nx.Graph, new_graph: nx.Graph, node1: int, node2: int, q: Queue, i: int):
    neighbors1 = [n for n in graph.neighbors(node1)]
    for n in neighbors1:
        if n != node2:
            atom_indices = graph.nodes[node1]["atom_indices"].union(graph.nodes[n]["atom_indices"])
            motif_smiles = fragment2smiles(mol, atom_indices)
            q.put((motif_smiles, i, -1))
    neighbors2 = [n for n in graph.neighbors(node2)]
    for n in neighbors2:
        if n != node1:
            atom_indices = graph.nodes[node2]["atom_indices"].union(graph.nodes[n]["atom_indices"])
            motif_smiles = fragment2smiles(mol, atom_indices)
            q.put((motif_smiles, i, -1))
    neighbors = [n for n in new_graph.neighbors(node1)]
    for n in neighbors:
        atom_indices = new_graph.nodes[node1]["atom_indices"].union(new_graph.nodes[n]["atom_indices"])
        motif_smiles = fragment2smiles(mol, atom_indices)
        q.put((motif_smiles, i, 1))

def apply_merging_operation_producer(motif: str, batch: List[MolGraph], q: Queue):
    [mol.apply_merging_operation_producer(motif, q) for mol in batch]
    q.put(None)

def apply_merging_operation_consumer(mols: List[MolGraph], stats: Dict[str, int], indices: Dict[str, Dict[int, int]], q: Queue, num_workers: int):
    num_tasks_done = 0
    while True:
        info = q.get()
        if info == None:
            num_tasks_done += 1
            if num_tasks_done == num_workers:
                break
        else:
            (motif, i, change) = info
            if isinstance(change, int):
                stats[motif] += change
                indices[motif][i] += change
            else:
                assert isinstance(change, nx.Graph)
                indices[motif][i] = 0
                mols[i].merging_graph = change

def apply_merging_operation(
    motif: str,
    mols: List[MolGraph],
    stats: Dict[str, int],
    indices: Dict[str, Dict[int, int]],
    num_workers: int = 1,
):
    mols_to_process = [mols[i] for i, freq in indices[motif].items() if freq > 0]
    if num_workers > 1:
        batch_size = (len(mols_to_process) -1 ) // num_workers + 1
        batches = [mols_to_process[i : i + batch_size] for i in range(0, len(mols_to_process), batch_size)]
        q = Queue()
        producers = [Process(target=apply_merging_operation_producer, args=(motif, batches[i], q)) for i in range(num_workers)]
        [p.start() for p in producers]
        apply_merging_operation_consumer(mols, stats, indices, q, num_workers)
        [p.join() for p in producers]
    else:
        [mol.apply_merging_operation(motif, stats, indices) for mol in mols_to_process]
    stats[motif] = 0

def merging_operation_learning(
    train_path: str,
    operation_path: str,
    num_iters: int,
    min_frequency: int,
    num_workers: int,
    mp_threshold: int,
):

    print(f"[{datetime.now()}] Learning merging operations from {train_path}.")
    print(f"Number of workers: {num_workers}. Total number of CPUs: {mp.cpu_count()}.\n")

    mols = load_mols(train_path, num_workers)
    stats, indices = get_stats(mols, num_workers)

    trace = []
    dir = os.path.split(operation_path)[0]
    os.makedirs(dir, exist_ok=True)
    output = open(operation_path, "w")
    for i in range(num_iters):
        print(f"[{datetime.now()}] Iteration {i}.")
        motif = max(stats, key=lambda x: (stats[x], x))
        if stats[motif] < min_frequency:
            print(f"No motif has frequency >= {min_frequency}. Stopping.\n")
            break
        print(f"[Iteration {i}] Most frequent motif: {motif}, frequency: {stats[motif]}.\n")
        trace.append((motif, stats[motif]))

        apply_merging_operation(
            motif = motif,
            mols = mols,
            stats = stats,
            indices = indices,
            num_workers = num_workers if stats[motif] >= mp_threshold else 1,
        )

        output.write(f"{motif}\n")

    output.close()
    print(f"[{datetime.now()}] Merging operation learning finished.")
    print(f"The merging operations are in {operation_path}.\n\n")

    return trace


In [12]:
import argparse
from arguments import parse_arguments

In [13]:
from model.mydataclass import Paths

In [14]:
import argparse # Just checking of the arguments.py file is imported and working
from arguments import parse_arguments
args = parse_arguments()
print(args)

Namespace(data_dir='/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/data/', preprocess_dir='/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/preprocess/', output_dir='/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/output/', tensorboard_dir='/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/tensorboard/', dataset='QM9/', job_name='train_micam', model_dir=None, generate_path='samples', num_workers=60, cuda=0, seed=2, num_operations=1000, num_iters=3000, min_frequency=0, mp_thd=100000.0, hidden_size=256, atom_embed_size=[192, 16, 16, 16, 16], edge_embed_size=256, motif_embed_size=[256, 256], latent_size=64, depth=15, motif_depth=6, dropout=0.3, virtual=False, pooling='add', steps=30000, batch_size=2000, lr=0.005, lr_anneal_iter=50, lr_anneal_rate=0.99, grad_clip_norm=1.0, beta_warmup=3000, beta_min=0.001, beta_max=0.3, beta_anneal_period=40000, prop_weight=0.2, num_sample=10000)


The following hyperparameters were changed:

parser.add_argument('--num_workers', type=int, default=60)  # Reduced number of workers

parser.add_argument('--num_iters', type=int, default=1000)  # Fewer iterations (previously 3000)  

parser.add_argument('--steps', type=int, default=10000)  # Fewer training steps (previously 50000)

parser.add_argument('--batch_size', type=int, default=32)  # Smaller batch size (previously 128) - tried changing this but gave an error

parser.add_argument('--dropout', type=float, default=0.2)  # Slightly lower dropout (previously 0.3)

parser.add_argument('--lr', type=float, default=1e-3)  # Lower learning rate (previously 1e5)

In [16]:
if __name__ == "__main__":

    args = parse_arguments()
    paths = Paths(args)

    learning_trace = merging_operation_learning(
        train_path = paths.train_path,
        operation_path = paths.operation_path,
        num_iters = args.num_iters,
        min_frequency = args.min_frequency,
        num_workers = args.num_workers,
        mp_threshold = args.mp_thd,
    )

Streaming output truncated to the last 5000 lines.

[2023-12-08 03:03:23.658734] Iteration 1335.
[Iteration 1335] Most frequent motif: CCCCCCCC/C=C\CCCCCCCN(C(=O)C(CCCCCC)CCCCCCCC)C(CC1CCCN1CC)C(=O)NC(C)(C)C, frequency: 1.

[2023-12-08 03:03:23.663409] Iteration 1336.
[Iteration 1336] Most frequent motif: CCCCCCCC/C=C\CCCCCCCN(C(=O)C(CCCCCC)CCCCCCCC)C(C(=O)NCCN1CCOCC1)c1cc(C)nn1C, frequency: 1.

[2023-12-08 03:03:23.667985] Iteration 1337.
[Iteration 1337] Most frequent motif: CCCCCCCC/C=C\CCCCCCCN(C(=O)C(CCCCCC)CCCCCCCC)C(C(=O)NCCN1CCOCC1)N1CCOCC1, frequency: 1.

[2023-12-08 03:03:23.672312] Iteration 1338.
[Iteration 1338] Most frequent motif: CCCCCCCC/C=C\CCCCCCCN(C(=O)C(CCCCCC)CCCCCCCC)C(C(=O)NCCN1CCOCC1)N1CCCCC1, frequency: 1.

[2023-12-08 03:03:23.676592] Iteration 1339.
[Iteration 1339] Most frequent motif: CCCCCCCC/C=C\CCCCCCCN(C(=O)C(CCCCCC)CCCCCCCC)C(C(=O)NCCN1CCOCC1)N(C)C, frequency: 1.

[2023-12-08 03:03:23.680961] Iteration 1340.
[Iteration 1340] Most frequent motif: CCCCC

# Motif Vocab Construction

In [15]:
pwd # just checking

'/content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/src'

In [16]:
from rdkit import Contrib
from rdkit.Contrib import SA_Score
from rdkit.Contrib.SA_Score import sascorer

In [17]:
import multiprocessing as mp
import os
import os.path as path
import pickle
from collections import Counter
from datetime import datetime
from functools import partial
from typing import List, Tuple

from tqdm import tqdm

from arguments import parse_arguments
from model.mol_graph import MolGraph
from model.mydataclass import Paths

In [18]:
def apply_operations(batch: List[Tuple[int, str]], mols_pkl_dir: str) -> Counter:
    vocab = Counter()
    pos = mp.current_process()._identity[0]
    with tqdm(total = len(batch), desc=f"Processing {pos}", position=pos-1, ncols=80, leave=False) as pbar:
        for idx, smi in batch:
            mol = MolGraph(smi, tokenizer="motif")
            with open(path.join(mols_pkl_dir, f"{idx}.pkl"), "wb") as f:
                pickle.dump(mol, f)
            vocab = vocab + Counter(mol.motifs)
            pbar.update()
    return vocab

def motif_vocab_construction(
    train_path: str,
    vocab_path: str,
    operation_path: str,
    num_operations: int,
    num_workers: int,
    mols_pkl_dir: str,
):

    print(f"[{datetime.now()}] Construcing motif vocabulary from {train_path}.")
    print(f"Number of workers: {num_workers}. Total number of CPUs: {mp.cpu_count()}.")

    data_set = [(idx, smi.strip("\n")) for idx, smi in enumerate(open(train_path))]
    batch_size = (len(data_set) - 1) // num_workers + 1
    batches = [data_set[i : i + batch_size] for i in range(0, len(data_set), batch_size)]
    print(f"Total: {len(data_set)} molecules.\n")

    print(f"Processing...")
    vocab = Counter()
    os.makedirs(mols_pkl_dir, exist_ok=True)
    MolGraph.load_operations(operation_path, num_operations)
    func = partial(apply_operations, mols_pkl_dir=mols_pkl_dir)
    with mp.Pool(num_workers, initializer=tqdm.set_lock, initargs=(mp.RLock(),)) as pool:
        for batch_vocab in pool.imap(func, batches):
            vocab = vocab + batch_vocab

    atom_list = [x for (x, _) in vocab.keys() if x not in MolGraph.OPERATIONS]
    atom_list.sort()
    new_vocab = []
    full_list = atom_list + MolGraph.OPERATIONS
    for (x, y), value in vocab.items():
        assert x in full_list
        new_vocab.append((x, y, value))

    index_dict = dict(zip(full_list, range(len(full_list))))
    sorted_vocab = sorted(new_vocab, key=lambda x: index_dict[x[0]])
    with open(vocab_path, "w") as f:
        for (x, y, _) in sorted_vocab:
            f.write(f"{x} {y}\n")

    print(f"\r[{datetime.now()}] Motif vocabulary construction finished.")
    print(f"The motif vocabulary is in {vocab_path}.\n\n")


In [21]:
if __name__ == "__main__":

    args = parse_arguments()
    paths = Paths(args)
    os.makedirs(paths.preprocess_dir, exist_ok=True)

    motif_vocab_construction(
        train_path = paths.train_path,
        vocab_path = paths.vocab_path,
        operation_path = paths.operation_path,
        num_operations = args.num_operations,
        mols_pkl_dir = paths.mols_pkl_dir,
        num_workers = args.num_workers,
    )

[2023-12-08 03:21:43.536856] Construcing motif vocabulary from /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/data/QM9/train.smiles.
Number of workers: 60. Total number of CPUs: 2.
Total: 1920 molecules.

Processing...
[2023-12-08 03:47:07.946256] Motif vocabulary construction finished.
The motif vocabulary is in /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/preprocess/QM9/num_ops_1000/vocab.txt.




# Make Training Data

In [19]:
import multiprocessing as mp
import os
import os.path as path
import pickle
from datetime import datetime
from functools import partial
from typing import List, Tuple

import torch
from tqdm import tqdm

from arguments import parse_arguments
from model.mol_graph import MolGraph
from model.mydataclass import Paths

In [20]:
def process_train_batch(batch: List[str], raw_dir: str, save_dir: str):
    pos = mp.current_process()._identity[0]
    with tqdm(total = len(batch), desc=f"Processing {pos}", position=pos-1, ncols=80, leave=False) as pbar:
        for file in batch:
            with open(path.join(raw_dir, file), "rb") as f:
                mol: MolGraph = pickle.load(f)
            data = mol.get_data()
            torch.save(data, path.join(save_dir, file.split()[0]+".pth"))
            pbar.update()

def process_valid_batch(batch: List[Tuple[int, str]], save_dir: str):
    pos = mp.current_process()._identity[0]
    with tqdm(total = len(batch), desc=f"Processing {pos}", position=pos-1, ncols=80, leave=False) as pbar:
        for idx, smi in batch:
            mol = MolGraph(smi, tokenizer="motif")
            data = mol.get_data()
            torch.save(data, path.join(save_dir, f"{idx}.pth"))
            pbar.update()

def make_trainig_data(
    mols_pkl_dir: str,
    valid_path: str,
    vocab_path: str,
    train_processed_dir: str,
    valid_processed_dir: str,
    vocab_processed_path: str,
    num_workers: int,
):

    print(f"[{datetime.now()}] Preprocessing traing data.")
    print(f"Number of workers: {num_workers}. Total number of CPUs: {mp.cpu_count()}.\n")


    print(f"[{datetime.now()}] Loading training set from {mols_pkl_dir}.\n")
    os.makedirs(train_processed_dir, exist_ok=True)
    data_set = os.listdir(mols_pkl_dir)
    batch_size = (len(data_set) - 1) // num_workers + 1
    batches = [data_set[i : i + batch_size] for i in range(0, len(data_set), batch_size)]
    func = partial(process_train_batch, raw_dir=mols_pkl_dir, save_dir=train_processed_dir)
    with mp.Pool(num_workers, initializer=tqdm.set_lock, initargs=(mp.RLock(),)) as pool:
        pool.map(func, batches)


    print(f"[{datetime.now()}] Preprocessing valid set from {valid_path}.\n")
    os.makedirs(valid_processed_dir, exist_ok=True)
    data_set = [(idx, smi.strip("\n")) for idx, smi in enumerate(open(valid_path))]
    batch_size = (len(data_set) - 1) // num_workers + 1
    batches = [data_set[i : i + batch_size] for i in range(0, len(data_set), batch_size)]
    func = partial(process_valid_batch, save_dir=valid_processed_dir)
    with mp.Pool(num_workers, initializer=tqdm.set_lock, initargs=(mp.RLock(),)) as pool:
        pool.map(func, batches)

    print(f"[{datetime.now()}] Preprocessing motif vocabulary from {vocab_path}.\n")
    vocab_data = MolGraph.preprocess_vocab()
    with open(vocab_processed_path, "wb") as f:
        torch.save(vocab_data, f)

    print(f"[{datetime.now()}] Preprocessing finished.\n\n")

In [21]:
from rdkit import Contrib
from rdkit.Contrib import SA_Score
from rdkit.Contrib.SA_Score import sascorer

In [22]:
if __name__ == "__main__":

    args = parse_arguments()
    paths  = Paths(args)

    MolGraph.load_operations(paths.operation_path, args.num_operations)
    MolGraph.load_vocab(paths.vocab_path)

    make_trainig_data(
        mols_pkl_dir = paths.mols_pkl_dir,
        valid_path = paths.valid_path,
        vocab_path = paths.vocab_path,
        train_processed_dir = paths.train_processed_dir,
        valid_processed_dir = paths.valid_processed_dir,
        vocab_processed_path = paths.vocab_processed_path,
        num_workers = args.num_workers,
    )


[2023-12-08 04:22:18.976153] Preprocessing traing data.
Number of workers: 60. Total number of CPUs: 2.

[2023-12-08 04:22:18.976422] Loading training set from /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/preprocess/QM9/num_ops_1000/mol_graphs.



Streaming output truncated to the last 5000 lines.













Processing 18:  22%|██████▎                      | 7/32 [01:52<02:42,  6.51s/it]










Processing 12:  22%|██████▎                      | 7/32 [01:53<02:54,  6.98s/it]











Processing 13:  22%|██████▎                      | 7/32 [01:53<02:51,  6.84s/it]








Processing 10:  22%|██████▎                      | 7/32 [01:53<02:53,  6.95s/it]






Processing 8:  25%|███████▌                      | 8/32 [01:54<02:14,  5.61s/it]












Processing 14:  25%|███████▎                     | 8/32 [01:54<02:15,  5.65s/it]















Processing 17:  25%|███████▎                     | 8/32 [01:54<02:10,  5.42s/it]









Processing 11:  25%|███████▎                     | 8/32 [01:55<02:25,  6.06s/it]

















Processing 19:  22%|██████▎                      | 7/32 [01:54<02:48,  6.75s/it]







Processing 9:  25%|███████▌                      | 8/32 [01:54<02:12,  5.50s/it]













Processing 15:  25

[2023-12-08 04:25:40.918917] Preprocessing valid set from /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/data/QM9/valid.smiles.

[2023-12-08 04:29:46.234765] Preprocessing motif vocabulary from /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/preprocess/QM9/num_ops_1000/vocab.txt.



100%|██████████| 374/374 [00:04<00:00, 91.62it/s] 


[2023-12-08 04:29:50.509713] Preprocessing finished.




# Training MiCaM

In [22]:
import argparse
import logging
import os
import os.path as path
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader

from arguments import parse_arguments
from model.dataset import MolsDataset, batch_collate
from model.MiCaM_VAE import MiCaM, VAE_Output
from model.mol_graph import MolGraph
from model.mydataclass import ModelParams, Paths, TrainingParams
from model.scheduler import beta_annealing_schedule

In [27]:
def train(args: argparse.Namespace):

    torch.manual_seed(args.seed)
    random.seed(args.seed)
    paths = Paths(args)
    tb = SummaryWriter(log_dir=paths.tensorboard_dir)

    model_params = ModelParams(args)
    training_params = TrainingParams(args)

    MolGraph.load_operations(paths.operation_path)
    MolGraph.load_vocab(paths.vocab_path)

    os.makedirs(paths.output_dir)
    log_file = path.join(paths.output_dir, "train.log")
    print(f"See {log_file} for log." )
    logging.basicConfig(
        filename = log_file,
        filemode = "w",
        format = "[%(asctime)s]: %(message)s",
        level = logging.INFO
    )

    model = MiCaM(model_params)
    optimizer = optim.Adam(model.parameters(), lr=training_params.lr)

    total_step, beta = 0, training_params.beta_min

    logging.info("HyperParameters:")
    logging.info(model_params)
    logging.info(training_params)

    scheduler = lr_scheduler.ExponentialLR(optimizer, training_params.lr_anneal_rate)
    beta_scheduler = beta_annealing_schedule(params=training_params, init_beta=beta, init_step=total_step)
    train_dataset = MolsDataset(paths.train_processed_dir)

    logging.info(f"Begin training...")
    os.makedirs(paths.model_save_dir)
    stop_train = False
    while True:
        for input in DataLoader(dataset=train_dataset, batch_size=training_params.batch_size, shuffle=True, collate_fn=batch_collate):
            total_step += 1
            model.zero_grad()

            input = input
            output: VAE_Output = model(input, beta=beta, prop_weight=training_params.prop_weight)

            output.total_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), training_params.grad_clip_norm)

            optimizer.step()
            output.log_tb_results(total_step, tb, beta, scheduler.get_last_lr()[0])

            if total_step % 50 == 0:
                output.print_results(total_step, lr=scheduler.get_last_lr()[0], beta=beta)

            if total_step % training_params.lr_anneal_iter == 0:
                scheduler.step()

            beta = beta_scheduler.step()

            if total_step == training_params.steps:
                stop_train = True
                break

        if stop_train: break

    model.eval()
    model.zero_grad()
    torch.cuda.empty_cache()
    model_path = path.join(paths.model_save_dir,"model.ckpt")
    motifs_embed_path = path.join(paths.model_save_dir,"motifs_embed.ckpt" )
    with torch.no_grad():
        ckpt = (model.state_dict(), optimizer.state_dict(), total_step, beta)
        torch.save(ckpt, model_path)
        model.save_motifs_embed(motifs_embed_path)
    tb.close()

In [28]:
if __name__ == "__main__":

    args = parse_arguments()

    train(args)

See /content/drive/MyDrive/AGILE2/AI4Sci-MiCaM/output/12-08/05:14:59-train_micam/train.log for log.


KeyError: ignored